# Introduction

Ce document résume les jeu de données, méthodologie, et statistiques utilisées pour l'estimation de la souffrance contenue dans les boîtes d'oeufs.

Nous commençons par l'import de la base de données complète d'open food facts obtenue le 31 mars 2025.

De cette base de données, nous ne retenons que les colonnes (goodcol) nécessaires au calcul du poids de souffrance, telles que définies dans le code.




In [1]:
import sys
import duckdb
sys.path.append("backend")

import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 1000)


In [2]:


minicol=['code', # ID
 'categories_tags', # déjà présent
 'labels_tags', # déjà présent
 'product_name', # déjà présent
 'generic_name',
 'quantity',
 'product_quantity_unit',
 'product_quantity',
 'allergens_tags',
 'ingredients_tags',
 'ingredients',
 'countries_tags',
 'images',
 ]

duckdb.execute(f"CREATE OR REPLACE VIEW db_col AS SELECT {','.join(minicol)} FROM 'C:/Users/DELL/Desktop/Data4Good local/food.parquet'")
duckdb.execute("SUMMARIZE db_col").df()

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,code,VARCHAR,,9999999999999,3493842,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,0.00
1,categories_tags,VARCHAR[],[],[zh:黑芝麻醬],188438,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,53.46
2,labels_tags,VARCHAR[],[],[zh:饼干],106922,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,60.97
3,product_name,"STRUCT(lang VARCHAR, ""text"" VARCHAR)[]",[],"[{'lang': zh, 'text': 風味發酵乳}]",2517162,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,0.00
4,generic_name,"STRUCT(lang VARCHAR, ""text"" VARCHAR)[]",[],"[{'lang': zh, 'text': 鹽水香蕉蕾}]",122360,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,0.00
5,quantity,VARCHAR,,😐,64588,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,59.46
6,product_quantity_unit,VARCHAR,%,mmol/l,4,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,73.96
7,product_quantity,VARCHAR,0,999999999999999999,7534,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,67.81
8,allergens_tags,VARCHAR[],[],"[zh:乳制品, zh:小麦, zh:豆制品]",13530,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,0.69
9,ingredients_tags,VARCHAR[],[],"[zu:salt, zu:anti-caking-agent, zu:potassium-iodate]",882416,<NA>,<NA>,<NA>,<NA>,<NA>,3864051,71.01


# Identification des oeufs

## Approche additive

Le résumé de la base de données indique qu'il y a 3,78 millions de produits.

On note au passage que `quantity` et `product_quantity`, qui constituent la base de notre approche, nécessiteront un gros travail se nettoyage.

L'identification des boîtes d'oeufs semble assez pédestre, puisque selon l'approche taxonomique il suffirait de sélectionner les produits ayant `en:chicken-eggs` dans `categories_tags`.

Combien y en a-t-il ?


In [3]:
duckdb.execute("SELECT COUNT(*) FROM db_col WHERE 'en:chicken-eggs' IN categories_tags").df()


,count_star()
0,4321


## Approche soustractive

On suppose que la taxonomie est incomplète / imparfaite et on tente une approche par soustraction:
plutôt que de prendre les éléments d'oeufs de poule, on sélectionne les oeufs dont on enlève tout ce qui est identifié comme oeufs d'autre animaux, en supposant que le défaut est oeuf de poule.

Afin d'arbitrer entre les deux approches, on compare le nombre d'éléments de cette approche avec le nombre d'éléments précédents et échantillonne quelques éléments afin de voir si cela a du sens.


In [4]:
ltags=duckdb.execute("SELECT list_distinct(categories_tags) FROM db_col WHERE 'en:eggs' IN categories_tags").df()
set(x for xs in ltags.iloc[:, 0] for x in xs if x.startswith("en") and x.endswith("-eggs"))


{'en:barn-chicken-eggs',
 'en:boiled-eggs',
 'en:british-free-range-eggs',
 'en:brown-eggs',
 'en:cage-chicken-eggs',
 'en:caged-chicken-eggs',
 'en:century-eggs',
 'en:chicken-eggs',
 'en:chocolate-eggs',
 'en:duck-eggs',
 'en:easter-eggs',
 'en:farming-products-eggs',
 'en:filled-chocolate-eggs',
 'en:fish-and-meat-and-eggs',
 'en:fish-eggs',
 'en:free-range-chicken-eggs',
 'en:free-range-duck-eggs',
 'en:free-range-large-eggs',
 'en:free-range-organic-large-chicken-eggs',
 'en:fresh-chicken-eggs',
 'en:fresh-eggs',
 'en:frozen-chicken-eggs',
 'en:frozen-eggs',
 'en:grade-a-eggs',
 'en:grade-aa-eggs',
 'en:hard-cooked-peeled-eggs',
 'en:labeled-eggs',
 'en:large-chicken-eggs',
 'en:large-eggs',
 'en:large-free-run-chicken-eggs',
 'en:large-organic-chicken-eggs',
 'en:large-organic-free-range-chicken-eggs',
 'en:medium-organic-free-range-chicken-eggs',
 'en:organic-chicken-eggs',
 'en:organic-eggs',
 'en:organic-free-range-chicken-eggs',
 'en:organic-large-brown-chicken-eggs',
 'en:pi

Avec cette approche nous ne sommes pas parvenus à retrouver les éléments "ostrich eggs", "guineafowl eggs", etc.
Nous parvenons à cet ensemble d'éléments à exclure :

In [5]:
pas_poule={'en:chocolate-eggs',
 'en:duck-eggs',
 'en:easter-eggs',
 'en:fish-eggs',
 'en:free-range-duck-eggs',
 'en:quail-eggs',
 'en:raw-quail-eggs',
 'en:savoury-eggs',
 'en:scotch-eggs',
 'en:streamed-eggs',
'en:meals',
'en:snacks',        
'en:meats-and-their-products',
'en:breads'
          }


joint="' NOT IN categories_tags AND '".join(list(pas_poule))
request="CREATE OR REPLACE VIEW eggs AS SELECT * FROM db_col WHERE 'en:eggs' IN categories_tags AND '" +\
         joint+"' NOT IN categories_tags"
#print(request)

duckdb.execute(request)
eggs_from_parquet_duckdb=duckdb.execute("FROM eggs").df()

In [6]:
eggs_from_parquet_duckdb.sample(50, random_state=10)

,code,categories_tags,labels_tags,product_name,generic_name,quantity,product_quantity_unit,product_quantity,allergens_tags,ingredients_tags,ingredients,countries_tags,images
1724,3760165984078,"[en:farming-products, en:eggs, en:chicken-eggs, en:free-range-chicken-eggs]","[en:organic, en:eu-organic, en:fr-bio-10, fr:ab-agriculture-biologique]","[{'lang': 'main', 'text': '6 oeufs bio plein air'}, {'lang': 'fr', 'text': '6 oeufs bio plein air'}]",[],,None,None,[],<NA>,None,[en:france],"[{'key': 'front_fr', 'imgid': 1, 'rev': 4, 'sizes': {'100': {'h': 56, 'w': 100}, '200': {'h': 112, 'w': 200}, '400': {'h': 225, 'w': 400}, 'full': {'h': 1152, 'w': 2050}}, 'uploaded_t': None, 'uploader': None}, {'key': '2', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 56, 'w': 100}, '200': None, '400': {'h': 225, 'w': 400}, 'full': {'h': 1153, 'w': 2050}}, 'uploaded_t': 1546185531, 'uploader': 'kiliweb'}, {'key': 'ingredients_fr', 'imgid': 2, 'rev': 7, 'sizes': {'100': {'h': 56, 'w': 100}, '200': {'h': 112, 'w': 200}, '400': {'h': 225, 'w': 400}, 'full': {'h': 1153, 'w': 2050}}, 'uploaded_t': None, 'uploader': None}, {'key': '1', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 56, 'w': 100}, '200': None, '400': {'h': 225, 'w': 400}, 'full': {'h': 1152, 'w': 2050}}, 'uploaded_t': 1546185529, 'uploader': 'kiliweb'}]"
4969,8480010200683,"[en:farming-products, en:eggs]","[en:organic, en:eu-organic, en:es-eco-025-na, en:green-dot, en:nutriscore]","[{'lang': 'main', 'text': 'Huevos'}, {'lang': 'es', 'text': 'Huevos'}]",[],,None,None,[],<NA>,None,[en:spain],"[{'key': 'nutrition_es', 'imgid': 3, 'rev': 9, 'sizes': {'100': {'h': 78, 'w': 100}, '200': {'h': 156, 'w': 200}, '400': {'h': 313, 'w': 400}, 'full': {'h': 1200, 'w': 1534}}, 'uploaded_t': None, 'uploader': None}, {'key': 'front_es', 'imgid': 1, 'rev': 15, 'sizes': {'100': {'h': 75, 'w': 100}, '200': {'h': 150, 'w': 200}, '400': {'h': 300, 'w': 400}, 'full': {'h': 901, 'w': 1200}}, 'uploaded_t': None, 'uploader': None}, {'key': '2', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 85, 'w': 100}, '200': None, '400': {'h': 341, 'w': 400}, 'full': {'h': 1200, 'w': 1407}}, 'uploaded_t': 1627922756, 'uploader': 'kiliweb'}, {'key': '1', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 100, 'w': 75}, '200': None, '400': {'h': 400, 'w': 300}, 'full': {'h': 1200, 'w': 901}}, 'uploaded_t': 1627922755, 'uploader': 'kiliweb'}, {'key': '3', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 78, 'w': 100}, '200': None, '400': {'h': 313, 'w': 400}, 'full': {'h': 1200, 'w': 1534}}, 'uploaded_..."
5302,4056489000792,"[en:farming-products, en:eggs, en:chicken-eggs, en:free-range-chicken-eggs]","[en:british-lion-quality, en:rspca-assured]","[{'lang': 'main', 'text': 'Free Range Very Large Eggs'}, {'lang': 'en', 'text': 'Free Range Very Large Eggs'}]",[],6 Eggs,None,0.0,[en:eggs],"[en:free-range-eggs, en:egg]","[{""percent_max"":100.0,""percent_min"":100.0,""is_in_taxonomy"":1,""percent_estimate"":100.0,""vegan"":""no"",""id"":""en:free-range-eggs"",""text"":""Free range eggs"",""vegetarian"":""yes"",""ciqual_food_code"":""22000"",""percent"":null,""from_palm_oil"":null,""ingredients"":null,""ecobalyse_code"":""egg-indoor-code3"",""processing"":null,""labels"":null,""origins"":null,""ecobalyse_proxy_code"":null,""quantity"":null,""quantity_g"":null,""ciqual_proxy_food_code"":null}]",[en:united-kingdom],"[{'key': 'ingredients_en', 'imgid': 1, 'rev': 5, 'sizes': {'100': {'h': 75, 'w': 100}, '200': {'h': 150, 'w': 200}, '400': {'h': 300, 'w': 400}, 'full': {'h': 3072, 'w': 4096}}, 'uploaded_t': None, 'uploader': None}, {'key': '2', 'imgid': None, 'rev': None, 'sizes': {'100': {'h': 76, 'w': 100}, '200': None, '400': {'h': 302, 'w': 400}, 'full': {'h': 1889, 'w': 2501}}, 'uploaded_t': 1740322635, 'uploader': 'nah'}, {'key': 'nutrition_en', 'imgid': 2, 'rev': 16, 'sizes': {'100': {'h': 90, 'w': 100}, '200': {'h': 180, 'w': 200}, '400': {'h': 360, 'w': 400}, 'full': {'h': 1490, 'w': 1656}}, 'uploaded_t': None, 'u

Conversion de la synthaxe duckdb en json

In [7]:
import json
import numpy as np

cols_to_json = []

for col in eggs_from_parquet_duckdb.columns:
    sample = eggs_from_parquet_duckdb[col].dropna().head(20)
    if sample.apply(lambda x: isinstance(x, (list, dict, np.ndarray))).any():
        cols_to_json.append(col)

cols_to_json

cols_to_json_for_import = cols_to_json + ['ingredients']

In [8]:
eggs_from_parquet = eggs_from_parquet_duckdb.copy()

def ndarray_to_json(arr):
    if isinstance(arr, (list, dict)):
        return json.dumps(arr)
    elif isinstance(arr, np.ndarray):
        return json.dumps(arr.tolist())
    else:
        return arr  # valeur non traitée

for col in cols_to_json_for_import:
    print(col)
    print(type(eggs_from_parquet['categories_tags'][0]))
    print(eggs_from_parquet['categories_tags'][0])
    eggs_from_parquet[col] = eggs_from_parquet_duckdb[col].apply(ndarray_to_json)
    print(type(eggs_from_parquet['categories_tags'][0]))
    print(eggs_from_parquet['categories_tags'][0])
    print()


categories_tags
<class 'numpy.ndarray'>
['en:farming-products' 'en:eggs']
<class 'str'>
["en:farming-products", "en:eggs"]

labels_tags
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

product_name
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

generic_name
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

allergens_tags
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

ingredients_tags
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

countries_tags
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

images
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-products", "en:eggs"]

ingredients
<class 'str'>
["en:farming-products", "en:eggs"]
<class 'str'>
["en:farming-pro

In [9]:
with open("../data/cols_to_json.txt", "w") as f:
    json.dump(cols_to_json_for_import, f)

eggs_from_parquet.to_csv("../data/eggs_from_parquet.csv", index=False)
eggs_from_parquet

,code,categories_tags,labels_tags,product_name,generic_name,quantity,product_quantity_unit,product_quantity,allergens_tags,ingredients_tags,ingredients,countries_tags,images
0,00003100,"[""en:farming-products"", ""en:eggs""]",[],"[{""lang"": ""main"", ""text"": ""Hard Boiled Eggs""}, {""lang"": ""fr"", ""text"": ""Hard Boiled Eggs""}]",[],2,None,0.0,"[""en:eggs""]","[""fr:eggs"", ""en:e330"", ""fr:sodium-benzoate"", ""fr:nisin-preparation""]","[{""percent_max"":100.0,""percent_min"":100.0,""is_in_taxonomy"":0,""percent_estimate"":100.0,""vegan"":null,""id"":""fr:eggs"",""text"":""Eggs"",""vegetarian"":null,""ciqual_food_code"":null,""percent"":null,""from_palm_oil"":null,""ingredients"":[{""percent_max"":100.0,""percent_min"":25.0,""is_in_taxonomy"":0,""percent_estimate"":62.5,""vegan"":null,""id"":""fr:eggs"",""text"":""Eggs"",""vegetarian"":null,""ciqual_food_code"":null,""percent"":null,""from_palm_oil"":null,""ingredients"":null,""ecobalyse_code"":null,""processing"":null,""labels"":null,""origins"":null,""ecobalyse_proxy_code"":null,""quantity"":null,""quantity_g"":null,""ciqual_proxy_food_code"":null},{""percent_max"":50.0,""percent_min"":0.0,""is_in_taxonomy"":1,""percent_estimate"":18.75,""vegan"":""yes"",""id"":""en:e330"",""text"":""Citric Acid"",""vegetarian"":""yes"",""ciqual_food_code"":null,""percent"":null,""from_palm_oil"":null,""ingredients"":null,""ecobalyse_code"":null,""processing"":null,""labels"":null,""origins"":null,""ecobalyse_proxy_code"":null,""quantity"":null,""quantity_g"":null,""ciqual_proxy_food_code"":null}...","[""en:france""]","[{""key"": ""front"", ""imgid"": 1, ""rev"": 3, ""sizes"": {""100"": {""h"": 100, ""w"": 75}, ""200"": {""h"": 200, ""w"": 150}, ""400"": {""h"": 400, ""w"": 300}, ""full"": {""h"": 2666, ""w"": 2000}}, ""uploaded_t"": null, ""uploader"": null}, {""key"": ""nutrition_fr"", ""imgid"": 3, ""rev"": 18, ""sizes"": {""100"": {""h"": 100, ""w"": 85}, ""200"": {""h"": 200, ""w"": 170}, ""400"": {""h"": 400, ""w"": 340}, ""full"": {""h"": 785, ""w"": 668}}, ""uploaded_t"": null, ""uploader"": null}, {""key"": ""1"", ""imgid"": null, ""rev"": null, ""sizes"": {""100"": {""h"": 100, ""w"": 75}, ""200"": null, ""400"": {""h"": 400, ""w"": 300}, ""full"": {""h"": 2666, ""w"": 2000}}, ""uploaded_t"": 1415119256, ""uploader"": ""openfoodfacts-contributors""}, {""key"": ""ingredients_en"", ""imgid"": 3, ""rev"": 22, ""sizes"": {""100"": {""h"": 16, ""w"": 100}, ""200"": {""h"": 31, ""w"": 200}, ""400"": {""h"": 63, ""w"": 400}, ""full"": {""h"": 98, ""w"": 624}}, ""uploaded_t"": null, ""uploader"": null}, {""key"": ""2"", ""imgid"": null, ""rev"": null, ""sizes"": {""100"": {""h"": 100, ""w"": 70}, ""200"": null, ""400"": {""h"": 400, ""w"": 278}, ""full"": {""h"": 1002,..."
1,0011110797698,"[""en:farming-products"", ""en:eggs"", ""en:undefined""]",<NA>,"[{""lang"": ""main"", ""text"": ""Natural Grade Aa Large Brown Eggs""}, {""lang"": ""en"", ""text"": ""Natural Grade Aa Large Brown Eggs""}]",[],50 g,g,50.0,[],"[""en:large-brown-eggs""]","[{""percent_max"":100.0,""percent_min"":100.0,""is_in_taxonomy"":0,""percent_estimate"":100.0,""vegan"":null,""id"":""en:large-brown-eggs"",""text"":""LARGE BROWN EGGS"",""vegetarian"":null,""ciqual_food_code"":null,""percent"":null,""from_palm_oil"":null,""ingredients"":null,""ecobalyse_code"":null,""processing"":null,""labels"":null,""origins"":null,""ecobalyse_proxy_code"":null,""quantity"":null,""quantity_g"":null,""ciqual_proxy_food_code"":null}]","[""en:united-states""]","[{""key"": ""nutrition_en"", ""imgid"": 2, ""rev"": 6, ""sizes"": {""100"": {""h"": 100, ""w"": 43}, ""200"": {""h"": 200, ""w"": 86}, ""400"": {""h"": 400, ""w"": 172}, ""full"": {""h"": 1200, ""w"": 515}}, ""uploaded_t"": null, ""uploader"": null}, {""key"": ""front_en"", ""imgid"": 1, ""rev"": 4, ""sizes"": {""100"": {""h"": 100, ""w"": 45}, ""200"": {""h"": 200, ""w"": 89}, ""400"": {""h"": 400, ""w"": 178}, ""full"": {""h"": 1200, ""w"": 534}}, ""uploaded_t"": null, ""up

In [10]:
head_db = duckdb.execute(f"SELECT * FROM 'C:/Users/DELL/Desktop/Data4Good local/food.parquet' limit 5").df()
